## Project Objective 
 - Stock market data can be interesting to analyze and as a further incentive, strong predictive models can have large financial payoff. The amount of financial data on the web is seemingly endless. A    large and well structured dataset on a wide array of companies can be hard to come by. In this project I have used a dataset with historical stock prices (5 years) for all companies           currently found on the S&P 500 index.
  Action Items -
  - In this project I have run multiple regression models to identify the best model to fit the dataset.  
   - I have downloaded the S&P 500 stocks from 2013 to 2018
   - Performed Exploratory Data Analysis (EDA) on the Facebook, Amazon, Apple, Netflix and Google (FAANG) stocks
   - I have plotted the yearly performance for the FAANG stocks
   - The key parameters have been explained using graphical representation of data.
  - I have run multiple regression models and compared them to find the best model for this dataset
   - For the best model I evaluated on a test dataset and report the r2 value and represent it visually.

In [2]:
try:
  displayHTML('<img src="https://media.giphy.com/media/10MEBgSHIhglMY/giphy.gif"/>') 
  #displayHTML('<img src="https://media.giphy.com/media/ycBC2muuv1yBlCL6Mn/giphy.gif"/>') 
except Exception as e:
  print("Gif load failed")

###Installing Libraries that we are going to use in this notebook

In [4]:
%sh
pip install plotly


Requirement already satisfied: plotly in /databricks/python3/lib/python3.7/site-packages (4.4.1)
Requirement already satisfied: retrying>=1.3.3 in /databricks/python3/lib/python3.7/site-packages (from plotly) (1.3.3)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from plotly) (1.12.0)
You are using pip version 19.0.3, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [5]:
import plotly.graph_objects as go
import plotly as py
from pyspark.sql.functions import unix_timestamp
import numpy as np

### Download the data for S&P 500 stocks from 2013 to 2018 
 - Data source : Downloaded dataset from Kaggle.
 link - https://www.kaggle.com/camnugent/sandp500/download
 - Alternatively, same data can be scraped from google Finance.
 link - https://www.google.com/finance

In [7]:
%sh
wget 'https://www.dropbox.com/s/f2oetiwm4iswhus/all_stocks_5yr.csv?dl=0' -O sp500faang.csv
  

--2019-12-14 04:47:36-- https://www.dropbox.com/s/f2oetiwm4iswhus/all_stocks_5yr.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/f2oetiwm4iswhus/all_stocks_5yr.csv [following]
--2019-12-14 04:47:36-- https://www.dropbox.com/s/raw/f2oetiwm4iswhus/all_stocks_5yr.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucce4829e0fe9ba10165b9a00b0c.dl.dropboxusercontent.com/cd/0/inline/AuNJzngRH4-JPQ0zItrXlLzHNSR_wO_KP7SqtRPgm_YL1lgbF2jSBKLq2G1APRLXndQudVlU8I38lXpjKzXrvU4pOGfFYOyQxCur5dblf_kLOYxF1EeM42YYheLr4nDA1NE/file# [following]
--2019-12-14 04:47:36-- https://ucce4829e0fe9ba10165b9a00b0c.dl.dropboxusercontent.com/cd/0/inline/AuNJzngRH4-JPQ0zItrXlLzHNSR_wO_KP7SqtRPgm_YL1lgbF2jSBKLq2G1APRLXndQudVlU8I38lXpjKzXrvU4pOGfFYOyQxCur5dblf_kLOYxF1EeM42YYheLr4nDA1NE/file
Resolving ucce4829e0fe9ba10165b9a00b0c.dl.dropboxusercontent.com (ucce4829e0fe9ba10165b9a00b0c.dl.dropboxusercontent.com)... 162.125.1.6, 2620:100:6016:6::a27d:106
Connecting to ucce4829e0fe9ba10165b9a00b0c.dl.dropboxusercontent.com (ucce4829e0fe9ba10165b9a00b0c.dl.dropboxusercontent.com)|162.125.1.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29580549 (28M) [text/plain]
Saving to: ‘sp500faang.csv’

 0K .......... .......... .......... .......... .......... 0% 1.12M 25s
 50K .......... .......... .......... .......... .......... 0% 1.94M 20s
 100K .......... .......... .......... .......... .......... 0% 1.61M 19s
 150K .......... .......... .......... .......... .......... 0% 4.64M 16s
 200K .......... .......... .......... .......... .......... 0% 1.12M 18s
 250K .......... .......... .......... .......... .......... 1% 1.07M 19s
 300K .......... .......... .......... .......... .......... 1% 7.66M 17s
 350K .......... .......... .......... .......... .......... 1% 1.04M 18s
 400K .......... .......... .......... .......... .......... 1% 6.12M 16s
 450K .......... .......... .......... .......... .......... 1% 1.19M 17s
 500K .......... .......... .......... .......... .......... 1% 6.69M 16s
 550K .......... .......... .......... .......... .......... 2% 1.08M 17s
 600K .......... .......... .......... .......... .......... 2% 6.28M 16s
 650K .......... .......... .......... .......... .......... 2% 1.09M 16s
 700K .......... .......... .......... .......... .......... 2% 6.36M 16s
 750K .......... .......... .......... .......... .......... 2% 3.10M 15s
 800K .......... .......... .......... .......... .......... 2% 6.58M 14s
 850K .......... .......... .......... .......... .......... 3% 1.09M 15s
 900K .......... .......... .......... .......... .......... 3% 1.08M 16s
 950K .......... .......... .......... .......... .......... 3% 8.07M 15s
 1000K .......... .......... .......... .......... .......... 3% 1.06M 15s
 1050K .......... .......... .......... .......... .......... 3% 8.65M 15s
 1100K .......... .......... .......... .......... .......... 3% 1.07M 15s
 1150K .......... .......... .......... .......... .......... 4% 5.61M 15s
 1200K .......... .......... .......... .......... .......... 4% 1.07M 15s
 1250K .......... .......... .......... .......... .......... 4% 7.55M 15s
 1300K .......... .......... .......... .......... .......... 4% 1.07M 15s
 1350K .......... .......... .......... .......... .......... 4% 8.03M 15s
 1400K .......... .......... .......... .......... .......... 5% 1.05M 15s
 1450K .......... .......... .......... .......... .......... 5% 7.06M 15s
 1500K .......... .......... .......... .......... .......... 5% 1.09M 15s
 1550K .......... .......... .......... .......... .......... 5% 5.78M 14s
 1600K .......... .......... .......... .......... .......... 5% 1.07M 15s
 1650K .......... .......... .......... .......... .......... 5% 7.70M 1

In [8]:
%sh
ls  #check the files in the directory

conf
derby.log
eventlogs
ganglia
logs
sp500faang.csv
test111.tsv
train111.tsv

## Data conversion and cleaning 
- Converting the downloaded csv file into pyspark dataframe
- Converting Date column to unix_timestamp
- Dropping the Null values
- Preparing the data by filtering the data for each FAANG stocks store in seperate dataframes

In [10]:
df_data = spark.read\
  .format("csv")\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load("file:/databricks/driver/sp500faang.csv")
df_data.show(3)

#test1.show(1)

+-------------------+-----+-----+-----+-----+-------+----+
 date| open| high| low|close| volume|Name|
+-------------------+-----+-----+-----+-----+-------+----+
2013-02-08 00:00:00|15.07|15.12|14.63|14.75|8407500| AAL|
2013-02-11 00:00:00|14.89|15.01|14.26|14.46|8882000| AAL|
2013-02-12 00:00:00|14.45|14.51| 14.1|14.27|8126000| AAL|
+-------------------+-----+-----+-----+-----+-------+----+
only showing top 3 rows

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.functions import date_format
from pyspark.sql.functions import col
from pyspark.sql.functions import udf,desc


#Creating a pyspark dataframe for future use
AAPLdf2 = df_data.where("Name = 'AAPL'")
AAPLdf3 = AAPLdf2.withColumn("date_converted", unix_timestamp("date", "yyyy-MM-dd").cast("double"))

split_col = F.split(AAPLdf3['date'], '-')
split_col2 = F.split(split_col.getItem(2),' ') 

startyear = 2013
AAPLdf3 = AAPLdf3.withColumn('DateofMon', split_col2.getItem(0))
AAPLdf3 = AAPLdf3.withColumn('Month', split_col.getItem(1))
AAPLdf3 = AAPLdf3.withColumn('Year', split_col.getItem(0)-startyear)

AAPLdf3 = AAPLdf3.withColumn('WeekSeq', ((AAPLdf3.Year)*12*4 + (AAPLdf3.Month-2)*4 + (AAPLdf3.DateofMon)/7 ).cast("int") )

#AAPLdf3 = AAPLdf3.withColumn('DayofWeek', ((AAPLdf3.DateofMon)%7).cast("int") )
#weekDay =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d').strftime('%w'))
#AAPLdf3 = AAPLdf3.withColumn('DayofWeek', weekDay(AAPLdf3['date']))

AAPLdf3.show(3)    


+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+
 date| open| high| low| close| volume|Name|date_converted|DateofMon|Month|Year|WeekSeq|
+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+
2013-02-08 00:00:00|67.7142|68.4014|66.8928|67.8542|158168416|AAPL| 1.3602816E9| 08| 02| 0.0| 1|
2013-02-11 00:00:00|68.0714|69.2771|67.6071|68.5614|129029425|AAPL| 1.3605408E9| 11| 02| 0.0| 1|
2013-02-12 00:00:00|68.5014|68.9114|66.8205|66.8428|151829363|AAPL| 1.3606272E9| 12| 02| 0.0| 1|
+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+
only showing top 3 rows

In [12]:
df_data.printSchema()


root
-- date: timestamp (nullable = true)
-- open: double (nullable = true)
-- high: double (nullable = true)
-- low: double (nullable = true)
-- close: double (nullable = true)
-- volume: integer (nullable = true)
-- Name: string (nullable = true)

Converting the date field to unix_timestamp format for feeding the data to the fitting model

In [14]:


df2 = df_data.withColumn("date_converted", unix_timestamp("date", "yyyy-MM-dd").cast("double"))
df2 = df2.withColumn("date_unix", unix_timestamp("date", "yyyy-MM-dd"))


#df.withColumn('total_col', df.a + df.b + df.c)

#split_date= split(df2['date'], '-')     
#df2= df2.withColumn('Year', split_date.getItem(0))
#df2= df2.withColumn('Month', split_date.getItem(1))
#df2= df2.withColumn('Day', split_date.getItem(2))

#df345 = df2.select("some_id", year(df2["date"]).alias('year'), month(df2["date"]).alias('month'), dayofmonth(df2["date"]).alias('day'),hour(df2["date"]).alias('hour')).show()
#df345.show(5)


md 
  ### Data cleaning 
  * Identifying the null values and dropping them
  * There were a few rows with null values that were dropped

In [16]:
df2.count()

Out[157]: 619040

In [17]:
df2 = df2.toPandas()
df2.isnull().sum()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning:

pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream

Out[158]: date 0
open 11
high 8
low 8
close 0
volume 0
Name 0
date_converted 0
date_unix 0
dtype: int64

In [18]:
df2 = df2.dropna()
df2.count()

Out[159]: date 619029
open 619029
high 619029
low 619029
close 619029
volume 619029
Name 619029
date_converted 619029
date_unix 619029
dtype: int64

# Data Explained

 We have filtered the data for each FAANG stocks , stored it in seperate dataframe and compared the performance of these stocks on yearly basis.
 
All the dataframes have the following columns: 

## Key fields - 
- Key fields - These are the features from the original dataset
* date - in format: yyyy-mm-dd
* open - price of the stock at market open (this is NYSE data so all in USD)
* high - Highest price reached in the day
* low  - Lowest price reached in the day
* close - price of the stock at market close (in USD) 
* volume - Number of shares traded
* Name - the stock's ticker name

Engineered Features
* We engineered the following features to improve our model. 
* avgof7D_lag3D   - running average of 7 Day with a lag of 3 days ( calculated from open price)
* avgof14D_lag7D  - running average of 14 Day with a lag of 7 days ( calculated from open price)
* avgof28D_lag14D - running average of 28 Day with a lag of 14 days ( calculated from open price)
* DateofMon - the date of a month as a number
* Month - the month as a number
* Year - the year as a number with Year 2013 as year 0. (at the start of the time line.)
* WeekSeq  - the week number starting with week 0 as the start of the time line. 
* date_converted - Since we are using this as a time series analysis, we are converting the date into a double format.  

## Predictor field - 

* We added the feature "avgof14D_lag0D" from the open value and we are trying to predict this smoothened value
- avgof14D_lag0D   - running average of 7 Day with a lag of 0 days ( calculated from open price)

We ended up using only the feature "14 Days with a lag of 7 days" among all the lag features, since we found that to be the most useful feature to get the best R2 value

## Detailed explanation of the reason to engineer new features

- In m1, we used randomsplit to split data into training and testing. This gave us a good r2 value.
- At that point we were not predicting. When we split data with time; the r2 value was close to 0.2 .( Poor fitting ) 
* * Training data: from 2013 to mid 2017 and testing data: from 2017 mid to 2018 mid; 
- This meant our model over trained on train data and was not doing well in predicting future (test data)
- We thought, the reason is the fluctuation of the stock price from day to day, the model was getting over trained.
- To mitigate this, we decided to take a moving average filter.
- To take moving average filter we used the "window" functionality in the pyspark.sql 
- We tried moving average filter for 7 days, 14 days and 28 days
- For our final results 14 days average was working better for us, so we used that here.
- We also added the following features. Dateofmonth, Month, Year, weekseq, date_converted
- The idea of using the date, month and year was to let the model develop and detect any patterns from monthly, weekly or yearly patterns. 
- These features helped improve the R2 model but only very slightly. ( 0.83 for APPL went to 0.85). 
- For predicting the stock values what really helped us was adding the stock values from 7 days back as a feature. This was from our domain knowledge of stock market, which is the essence of feature engineering.

In [20]:
## filter the data for each FAANG stocks store in seperate dataframes
AAPL = df2[(df2['Name'] == 'AAPL')]
GOOGL = df2[(df2['Name'] == 'GOOGL')]
FB = df2[(df2['Name'] == 'FB')]
AMZN = df2[(df2['Name'] == 'AMZN')]
NFLX = df2[(df2['Name'] == 'NFLX')]

# Visualization of data

In [22]:
#Plotting yearly performance of the FAANG stocks
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  
rcParams['figure.figsize'] = 10,10
fig = plt.figure()
#ax1 = fig.add_subplot(221)
ax1 = sns.lineplot(data=AMZN, x="date", y="open",label ='APPLE Stock Movement', markers=True, dashes=False)
ax1 = sns.lineplot(data=GOOGL, x="date", y="open",label ='GOOGLE Stock Movement', markers=True, dashes=False)
ax1 = sns.lineplot(data=AAPL, x="date", y="open",label ='APPLE Stock Movement', markers=True, dashes=False)
ax1 = sns.lineplot(data=FB, x="date", y="open",label ='FACEBOOK Stock Movement', markers=True, dashes=False)
ax1 = sns.lineplot(data=NFLX, x="date", y="open",label ='NETFLIX Stock Movement', markers=True, dashes=False)
fig.suptitle('Comparing Yearly Performance of the FAANG Stocks', fontsize=18)
display(fig)

In [23]:
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  
rcParams['figure.figsize'] = 10,7
fig = plt.figure()
#ax1 = fig.add_subplot(221)
ax1 = sns.lineplot(data=AMZN, x="date", y="open",label ='APPLE Stock Movement', markers=True, dashes=False)
ax1 = sns.lineplot(data=GOOGL, x="date", y="open",label ='GOOGLE Stock Movement', markers=True, dashes=False)
fig.suptitle('Comparing Yearly Performance high performing Stocks - Google and Apple', fontsize=18)
display(fig)

In [24]:
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  
rcParams['figure.figsize'] = 10,7
fig = plt.figure()
#ax1 = fig.add_subplot(221)
ax1 = sns.lineplot(data=AAPL, x="date", y="open",label ='APPLE Stock Movement', markers=True, dashes=False)
fig.suptitle('Yearly Performance of Apple stock of last 5 years', fontsize=18)
display(fig)

###Plotting data from different fields against 'date' parameter for Apple Stock to get a  better understanding of the fields.

In [26]:
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
fig = plt.figure()
ax1 = fig.add_subplot(221)
ax1 = sns.lineplot(x="date", y="open",
                  markers=True, dashes=False,label ='APPL', data=AAPL)
ax2 = fig.add_subplot(222, sharex=ax1, sharey=ax1)
ax2 = sns.lineplot(x="date", y="high",
                  markers=True, dashes=False,label ='APPL',data=AAPL)
ax3 = fig.add_subplot(223, sharex=ax1, sharey=ax1)
ax3 = sns.lineplot(x="date", y="low",
                  markers=True, dashes=False,label ='APPL', data=AAPL)
ax4 = fig.add_subplot(224, sharex=ax1, sharey=ax1)
ax4 = sns.lineplot(x="date", y="close",
                  markers=True, dashes=False,label ='APPL', data=AAPL)

display(fig)

In [27]:
# Taking a closer look at the Apple Stock trend from first week of January to first week of February in 2018
apple2018 = AAPL[(AAPL['date'] > '2018-01-05' )]
apple_m = apple2018.groupby(['date'])
agg_m = apple2018.aggregate({'open':np.mean, 'close':np.mean, 'high':np.mean, 'low':np.mean})
agg_m = apple2018.reset_index()

trace = go.Candlestick(x = agg_m['date'],
                       open = agg_m['open'],
                       high = agg_m['high'],
                       low = agg_m['low'],
                       close = agg_m['close']
                      )

data = [trace]

layout = {
    'title':'Closer look at Apple Stock trend',
    'xaxis': {'title':'Date',
             'rangeslider':{'visible':False}},
    'yaxis':{'title':'Price in US Dollars'}
}

fig_candle1 = go.Figure(data=data, layout=layout)
py.offline.iplot(fig_candle1)

Insights : The performance of the stock is seen to be improving during the second week of January 2018. Where as it gradually plummets towrards the end of the month.

##Feature Engineering
* Added more features from existing features
* We added the following features 
- avgof7D_lag3D   - running average of 7 Day with a lag of 3 days
- avgof14D_lag7D  - running average of 14 Day with a lag of 7 days
- avgof28D_lag14D - running average of 28 Day with a lag of 14 days

We ended up using only the 14 Days with a lag of 7 days, since we found that to be the most useful feature to get the best R2 value

In [30]:
# SMOOTHEN DATA
from pyspark.sql import functions as F
from pyspark.sql.window import Window

days = lambda i: i * 86400 # convert days to seconds. This is used in rangeBetween
df4 = AAPLdf3.withColumn('date', AAPLdf3.date.cast('timestamp'))

#create window by casting timestamp to long (number of seconds)
w7 = (Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(7), 0))
w14 = (Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(14), 0))
w28 = (Window.orderBy(F.col("date").cast('long')).rangeBetween(-days(28), 0))

df5 = df4.withColumn('avgof7D_lag3D', F.avg("open").over(w7))
df6 = df5.withColumn('avgof14D_lag7D', F.avg("open").over(w14))
df7 = df6.withColumn('avgof28D_lag14D', F.avg("open").over(w28))

df7.show(5)

# IN M1, WE USED RANDOMSPLIT TO SPLIT DATA INTO TRAINING AND TESTING
# THIS GAVE US A GOOD R2 VALUE.
# AT THAT POINT WE WERE NOT PREDICTING. 
# WHEN WE SPLIT DATA WITH TIME. TRAINING DATA: FROM 2013 TO MID 2017 AND TESTING DATA: FROM 2017 MID TO 2018 MID; 
# THE R2 VALUE WAS CLOSE TO 0.1 

# THIS MEANT OUR MODEL OVER TRAINED ON TRAIN DATA AND WAS NOT DOING WELL IN PREDICTING FUTURE (TEST DATA)
# WE THOUGHT, THE REASON IS THE FLUCTUATION OF THE STOCK PRICE FROM DAY TO DAY, THE MODEL WAS GETTING OVER TRAINED.

# TO MITIGATE THIS, WE DECIDED TO TAKE A MOVING AVERAGE FILTER
# TO TAKE MOVING AVERAGE FILTER WE USED THE "WINDOW" FUNCTIONALITY IN THE PYSPARK.SQL 
# WE TRIED MOVING AVERAGE FILTER FOR 7 DAYS, 14 DAYS AND 28 DAYS
# FOR OUR FINAL RESULTS 14 DAYS AVERAGE WAS WORKING BETTER FOR US, SO WE USED THAT HERE. 

# THE MOVING AVERAGE FILTER SMOOTHENED OUT THE DATA, BUT WE WERE STILL GETTING LOW R2 VALUE IN THE TEST DATA
# TO IMPROVE THE RESULT FURTHER WE THOUGHT OF SOME NEW IDEAS. 
# WE TRIED TO GET A DATASET WITH FINANCIAL NEWS, CONVERT TO SENTIMENT ANALYSIS ETC. 
# (DATA WAS AVAILBALE BUT IT WAS EXPENSIVE $125/MONTH SUBSCRIPTION WAS THE BEST WE COULD FIND. - EVEN FOR HISTORICAL DATA)
# NEXT WE TRIED TO BE MORE CREATIVE WITH FEATURE ENGINEERING, WITH DOMAIN KNOWLEDGE

+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+-----------------+-----------------+-----------------+
 date| open| high| low| close| volume|Name|date_converted|DateofMon|Month|Year|WeekSeq| avgof7D_lag3D| avgof14D_lag7D| avgof28D_lag14D|
+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+-----------------+-----------------+-----------------+
2013-02-08 00:00:00|67.7142|68.4014|66.8928|67.8542|158168416|AAPL| 1.3602816E9| 08| 02| 0.0| 1| 67.7142| 67.7142| 67.7142|
2013-02-11 00:00:00|68.0714|69.2771|67.6071|68.5614|129029425|AAPL| 1.3605408E9| 11| 02| 0.0| 1| 67.8928| 67.8928| 67.8928|
2013-02-12 00:00:00|68.5014|68.9114|66.8205|66.8428|151829363|AAPL| 1.3606272E9| 12| 02| 0.0| 1|68.09566666666666|68.09566666666666|68.09566666666666|
2013-02-13 00:00:00|66.7442|67.6628|66.1742|66.7156|118721995|AAPL| 1.3607136E9| 13| 02| 0.0| 1| 67.7578| 67.7578| 67.7578|
2013-02-14 00:00:00|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL| 1.3608E9| 14| 02| 0.0| 2| 67.47822| 67.47822| 67.47822|
+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+-----------------+-----------------+-----------------+
only showing top 5 rows

In [31]:
# INTRODUCE LAG TO DATA SO THAT OLD Y value CAN BE USED AS A FEATURE FOR FUTURE
# SMOOTHENING MOVES DATA TO FUTURE BY (N+1)/2 - standard fitler delay 

from pyspark.sql.functions import monotonically_increasing_id, lag
from pyspark.sql.window import Window

# Add ID to be used by the window function
df7 = df7.withColumn('id', monotonically_increasing_id())
# Set the window
w = Window.orderBy("id")
# Create the lagged value
value_lag = lag('avgof14D_lag7D',-7).over(w)
# Add the lagged values to a new column
df7 = df7.withColumn('avgof14D_lag0D', value_lag).dropna()
df7.show(15)

df_stg2 = df7

# TO IMPROVE OUR RESULTS WE DECIDED TO USE OUR DOMAIN KNOWLEDGE IN STOCK MARKET PREDICTION 
# THE BEST STOCK MARKET MODELS USE A TECHNIQUE CALLED LSTM (Long short-term memory), A "RECURRENT NEURAL NETWORK"
# WE COULD NOT FIGURE OUT A WAY TO USE LSTM WITH PYSPARK PIPELINE

# TO GET AROUND THIS 
# FOR PREDICITNG FUTURE STOCK VALUES, WE USE THE STOCK VALUES FROM 7 DAYS BACK AS A FEATURE
# WE USED A LAG FUNCTION TO CREATE A NEW FEATURE THAT DELAYS THE CURRENT DATA BY 7 DAYS. 
# FOR MORE EXPLANATION, LET US SHOW YOU TWO FIGURES
# FIRST ONE IS THE WHOLE DATA AND THEN THE ZOOMED IN VERSION 


+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+------------------+-----------------+-----------------+---+------------------+
 date| open| high| low| close| volume|Name|date_converted|DateofMon|Month|Year|WeekSeq| avgof7D_lag3D| avgof14D_lag7D| avgof28D_lag14D| id| avgof14D_lag0D|
+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+------------------+-----------------+-----------------+---+------------------+
2013-02-08 00:00:00|67.7142|68.4014|66.8928|67.8542|158168416|AAPL| 1.3602816E9| 08| 02| 0.0| 1| 67.7142| 67.7142| 67.7142| 0| 66.95315|
2013-02-11 00:00:00|68.0714|69.2771|67.6071|68.5614|129029425|AAPL| 1.3605408E9| 11| 02| 0.0| 1| 67.8928| 67.8928| 67.8928| 1| 66.59326666666666|
2013-02-12 00:00:00|68.5014|68.9114|66.8205|66.8428|151829363|AAPL| 1.3606272E9| 12| 02| 0.0| 1| 68.09566666666666|68.09566666666666|68.09566666666666| 2| 66.35179|
2013-02-13 00:00:00|66.7442|67.6628|66.1742|66.7156|118721995|AAPL| 1.3607136E9| 13| 02| 0.0| 1| 67.7578| 67.7578| 67.7578| 3| 66.06393|
2013-02-14 00:00:00|66.3599|67.3771|66.2885|66.6556| 88809154|AAPL| 1.3608E9| 14| 02| 0.0| 2| 67.47822| 67.47822| 67.47822| 4| 65.59707|
2013-02-15 00:00:00|66.9785|67.1656|65.7028|65.7371| 97924631|AAPL| 1.3608864E9| 15| 02| 0.0| 2| 67.39493333333333|67.39493333333333|67.39493333333333| 5| 65.15307|
2013-02-19 00:00:00|65.8714|66.1042|64.8356|65.7128|108854046|AAPL| 1.361232E9| 19| 02| 0.0| 2| 66.89108|67.17728571428572|67.17728571428572| 6| 64.82222|
2013-02-20 00:00:00|65.3842|65.3842|64.1142|64.1214|118891367|AAPL| 1.3613184E9| 20| 02| 0.0| 2| 66.26764| 66.95315| 66.95315| 7| 64.44337|
2013-02-21 00:00:00|63.7142|64.1671|63.2599|63.7228|111596821|AAPL| 1.3614048E9| 21| 02| 0.0| 3| 65.66164|66.59326666666666|66.59326666666666| 8| 63.85694000000001|
2013-02-22 00:00:00|64.1785|64.5142|63.7999|64.4014| 82583823|AAPL| 1.3614912E9| 22| 02| 0.0| 3| 65.22536| 66.35179| 66.35179| 9| 63.52552727272728|
2013-02-25 00:00:00|64.8356|65.0171|63.2242|63.2571| 92899597|AAPL| 1.3617504E9| 25| 02| 0.0| 3| 64.79678| 66.06393|66.21395454545454| 10|63.180200000000006|
2013-02-26 00:00:00|63.4028|64.5056|62.5228|64.1385|125096657|AAPL| 1.3618368E9| 26| 02| 0.0| 3| 64.56445000000001| 65.59707|65.97969166666665| 11|62.749163636363626|
2013-02-27 00:00:00|64.0614|64.6342|62.9499|63.5099|146674682|AAPL| 1.3619232E9| 27| 02| 0.0| 3| 64.26278333333333| 65.15307|65.83213076923076| 12| 62.53877272727272|
2013-02-28 00:00:00|63.4357|63.9814|63.0571|63.0571| 80532382|AAPL| 1.3620096E9| 28| 02| 0.0| 4| 63.93803333333333| 64.82222|65.66095714285714| 13|62.285527272727265|
2013-03-01 00:00:00|62.5714|62.5971|61.4257|61.4957|137899041|AAPL| 1.362096E9| 01| 03| 0.0| 4|63.747566666666664| 64.44337|65.45498666666667| 14|62.048518181818174|
+-------------------+-------+-------+-------+-------+---------+----+--------------+---------+-----+----+-------+------------------+-----------------+-----------------+---+------------------+
only showing top 15 rows

In [32]:
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  

pdata = df_stg2.toPandas()

rcParams['figure.figsize'] = 25, 10
fig = plt.figure()

ax1 = sns.lineplot(data=pdata, x="date", y="open", markers=True, dashes=False,label ='raw open')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg7", markers=True, dashes=False)
ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag0D", markers=True, dashes=False,label ='avgof14D_lag0D')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg28", markers=True, dashes=False)
ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag7D", markers=True, dashes=False,label ='avgof14D_lag7D')
display(fig)


In [33]:
import matplotlib.pyplot as plt  
from pylab import rcParams
import seaborn as sns  

data_zoom = df_stg2[(df_stg2['date'] > '2017-01-01' )]

pdata = data_zoom.toPandas()

rcParams['figure.figsize'] = 25, 10
fig = plt.figure()

ax1 = sns.lineplot(data=pdata, x="date", y="open", markers=True, dashes=False,label ='raw open')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg7", markers=True, dashes=False)
ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag0D", markers=True, dashes=False,label ='avgof14D_lag0D')
#ax1 = sns.lineplot(data=pdata, x="date", y="avg28", markers=True, dashes=False)
ax1 = sns.lineplot(data=pdata, x="date", y="avgof14D_lag7D", markers=True, dashes=False,label ='avgof14D_lag7D')
display(fig)

# THIS IS THE ZOOMED IN VERSION, THE BLUE IS THE RAW VALUES 
# THE ORANGE IS THE SMOOTHENED VERSION OF RAW ( SMOOTHENED OVER 14 DAYS) - "avgof14D_lag0D"
# WE ARE PREDICTING THIS NO LAG "avgof14D_lag0D"

# WE USE DATA FROM 7 DAYS BACK AS FEATURE "avgof14D_lag7D" ( delayed data)
# THIS REALLY HELPED US IMPROVE OUR R2 VALUE TO A RESPECTABLE 0.83 FOR APPL, 0.87 FOR FB, 0.93 FOR AMAZON, 0.85 FOR FB, AND 0.86 FOR GOOGLE. 
# WE RAN THE NOTE BOOK MULTIPLE TIMES TO GET THESE R2 VALUES. ( WITH DIFFERENT COMAPNIES DATAFRAME)

In [34]:
# MODEL CREATION IS DONE USING TRAINING DATA ONLY. 
# TEST RESULTS ARE CALCULATED BASED ON TEST DATA ONLY
from pyspark.sql.functions import unix_timestamp, lit

train_data = df_stg2.filter(df_stg2["date"] < unix_timestamp(lit('2017-07-01 00:00:00')).cast('timestamp'))
test_data  = df_stg2.filter(df_stg2["date"] > unix_timestamp(lit('2017-07-01 00:00:00')).cast('timestamp'))

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))


Number of training records: 1107
Number of testing records : 145

##Data modeling - Testing with one model
* Removed label and undesired columns
* Formatted data to fit RFormula format
* Tested pipeline and paramgrid for a single linear regression model
* CVfit with 3 fold cross validation
* Formula : avgof14D_lag0D ~ volume + date_converted + DateofMon + Month + Year + WeekSeq + avgof14D_lag7D

In [36]:
from pyspark.ml.feature import VectorAssembler,RFormula
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression, DecisionTreeRegressor
from pyspark.ml import Pipeline, Model
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 

columns = train_data.columns
# Not using Price (label) or address in features
columns.remove('date')
columns.remove('open')
columns.remove('high')
columns.remove('low')
columns.remove('close')
columns.remove('Name')
#columns.remove('market_value')
columns.remove('avgof7D_lag3D')
columns.remove('avgof14D_lag0D')
columns.remove('avgof28D_lag14D')
columns.remove('id')

formula = "{} ~ {}".format("avgof14D_lag0D", " + ".join(columns))
print("Formula : {}".format(formula))
rformula = RFormula(formula = formula)
lr = LinearRegression()
pipeline = Pipeline(stages=[rformula, lr])
# Parameter grid
paramGrid = ParamGridBuilder()\
          .addGrid(lr.regParam,[0.01, .04])\
          .build()
cv = CrossValidator()\
      .setEstimator(pipeline)\
      .setEvaluator(RegressionEvaluator()\
                       .setMetricName("r2"))\
      .setEstimatorParamMaps(paramGrid)\
      .setNumFolds(3)

cvModel = cv.fit(train_data)


Formula : avgof14D_lag0D ~ volume + date_converted + DateofMon + Month + Year + WeekSeq + avgof14D_lag7D
/databricks/spark/python/pyspark/ml/util.py:791: UserWarning:

Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.

## Setting up Pipeline for Multiple regression models
* Removed all columns other than converted date and volume
* Created framework for the models 
* * Linear regression, 
* * Generalized Linear regression and 
* * Decision tree regression
* Paramgrid setup for all three models
* Runs crossvalidation (cvfit) with 3-fold for all three models

In [38]:
#references
# https://stackoverflow.com/questions/45697720/with-pyspark-sql-functions-unix-timestamp-get-null
from pyspark.ml.feature import VectorAssembler,RFormula
from pyspark.ml.regression import LinearRegression, GeneralizedLinearRegression, DecisionTreeRegressor
from pyspark.ml import Pipeline, Model
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder 

columns = train_data.columns
# Not using Price (label) or address in features
columns.remove('date')
columns.remove('open')
columns.remove('high')
columns.remove('low')
columns.remove('close')
columns.remove('Name')
#columns.remove('market_value')
columns.remove('avgof7D_lag3D')
columns.remove('avgof14D_lag0D')
columns.remove('avgof28D_lag14D')
columns.remove('id')

# WE ARE PREDICTING "avgof14D_lag0D"
# OUR FORMULA FOR PIPELINE IS Formula : avgof14D_lag0D ~ volume + date_converted + avgof14D_lag7D
# WE ARE USING VOLUME, DATE_CONVERETED ( WHICH IS DATE IN SECONDS), AND THE LAG DATA. TO PREDICT THE FUTURE VALUES

formula = "{} ~ {}".format("avgof14D_lag0D", " + ".join(columns))
print("Formula : {}".format(formula))
rformula = RFormula(formula = formula)

# Pipeline basic to be shared across model fitting and testing
pipeline = Pipeline(stages=[])  # Must initialize with empty list!

# base pipeline (the processing here should be reused across pipelines)
basePipeline =[rformula]

#############################################################
# Specify Linear Regression model
lr = LinearRegression()
pl_lr = basePipeline + [lr]
pg_lr = ParamGridBuilder()\
          .baseOn({pipeline.stages: pl_lr})\
          .addGrid(lr.regParam,[0.01, .04])\
          .build()
#############################################################
# Specify Random Forrest model
rf = GeneralizedLinearRegression()
pl_rf = basePipeline + [rf]
pg_rf = ParamGridBuilder()\
      .baseOn({pipeline.stages: pl_rf})\
      .build()

#############################################################
# Specify Decision Tree model
dt = DecisionTreeRegressor()
pl_dt = basePipeline + [dt]
pg_dt = ParamGridBuilder()\
      .baseOn({pipeline.stages: pl_dt})\
      .build()

# One grid from the individual grids
paramGrid = pg_lr + pg_rf + pg_dt

Formula : avgof14D_lag0D ~ volume + date_converted + DateofMon + Month + Year + WeekSeq + avgof14D_lag7D

In [39]:
# The regression metric can be mse, rmse, mae, r2
# See the metrics here https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#regression-model-evaluation
# Should run more than 3 folds, but here we simplify so that it will complete
cv = CrossValidator()\
      .setEstimator(pipeline)\
      .setEvaluator(RegressionEvaluator()\
                       .setMetricName("r2"))\
      .setEstimatorParamMaps(paramGrid)\
      .setNumFolds(3)

cvModel = cv.fit(train_data)

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning:

Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.

In [40]:
import numpy as np
print("Best Model")
print(cvModel.getEstimatorParamMaps()[ np.argmax(cvModel.avgMetrics) ])
print("Worst Model")
print (cvModel.getEstimatorParamMaps()[ np.argmin(cvModel.avgMetrics) ])


Best Model
{Param(parent='Pipeline_3b19f6781196', name='stages', doc='a list of pipeline stages'): [RFormula_167fb6fc8d31, GeneralizedLinearRegression_25e8b4707d03]}
Worst Model
{Param(parent='Pipeline_3b19f6781196', name='stages', doc='a list of pipeline stages'): [RFormula_167fb6fc8d31, LinearRegression_1b673dbbbe02], Param(parent='LinearRegression_1b673dbbbe02', name='regParam', doc='regularization parameter (>= 0).'): 0.04}

In [41]:
#references
# https://stackoverflow.com/questions/45697720/with-pyspark-sql-functions-unix-timestamp-get-null


## Model evaluation
* Code necessary to plot r2 value of all three models
* Plotting of r2 value to compare models

In [43]:
import re
def paramGrid_model_name(m):
  params = [v for v in m.values() if type(v) is not list]
  name = [v[-1] for v in m.values() if type(v) is list][0]
  name = re.match(r'([a-zA-Z]*)', str(name)).groups()[0]
  return "{}{}".format(name,params)

# Resulting metric and model description
# get the measure from the CrossValidator, cvModel.avgMetrics
# get the model name & params from the paramGrid
# put them together here:
measures = zip(cvModel.avgMetrics, [paramGrid_model_name(m) for m in paramGrid])
metrics,model_names = zip(*measures)

In [44]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.clf() # clear figure
fig = plt.figure( figsize=(5, 5))
plt.style.use('fivethirtyeight')
axis = fig.add_axes([0.1, 0.3, 0.8, 0.6])
# plot the metrics as Y
#plt.plot(range(len(model_names)),metrics)
plt.bar(range(len(model_names)),metrics)
# plot the model name & param as X labels
plt.xticks(range(len(model_names)), model_names, rotation=70, fontsize=6)
plt.yticks(fontsize=6)
#plt.xlabel('model',fontsize=8)
plt.ylabel('R2 (higher is better)',fontsize=8)
plt.title('Model evaluations')
display(plt.show())

##Insight 
* Visual representation of comparison regression model based on r2 values. Out of all the models Decision Tree is identified as best model that fits this dataset.
* Based on the r2 value, all the three models were able to do well on the training data.

##Testing the best model
* Create test, train dataset
* Evaluate test data with best model

In [47]:
## Make predictions on test documents. 
#train_data, test_data  = AAPLdf3.randomSplit([0.6, 0.4], 24)   # proportions [], seed for random
# CrossValidator.fit() is in cvModel, which is the best model found.
predictions = cvModel.transform(test_data)
display(predictions.select('label', 'prediction').limit(100))

label,prediction
144.38600000000002,146.61178392504007
144.71200000000002,145.83467243958148
145.149,145.6886129810955
145.581,145.4205456396703
146.02636363636364,145.65571679005006
146.73636363636365,145.10941980866482
147.37,145.32117440784714
148.0681818181818,145.59977653780152
148.76727272727274,146.04644463511067
149.55090909090907,146.4723036530122


## Best model metrics 
* Display R2 value
* Display RMSE value

In [49]:
import sklearn.metrics
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

# Summarize the model over the training set and print out some metrics
print("Best pipeline", cvModel.bestModel.stages)
print("Best model", cvModel.bestModel.stages[1])

y_true = predictions.select('label').toPandas()
y_pred  = predictions.select('prediction').toPandas()

r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Best pipeline [RFormula_167fb6fc8d31, GeneralizedLinearRegression_25e8b4707d03]
Best model GeneralizedLinearRegression_25e8b4707d03
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning:

pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning:

pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream

r2_score: 0.8546742657416344
Root Mean Squared Error (RMSE) on test data = 3.39252

## Best model visualizaton ( y_true vs y_pred )
* Plot true values and predicated values
* Since plot is linear, we infer that model is good.

In [51]:
fig = plt.figure( figsize=(5, 5))
plt.scatter(y_true,y_pred)
plt.xlabel('y_true',fontsize=8)
plt.ylabel('y_pred ',fontsize=8)
plt.title('Best model visualization')
display(fig)

##Prediction (or discovery)

* For the best model I have evaluated on a test dataset and reported the r2 value and represented it visually.
* I USEd DATA FROM 7 DAYS BACK AS FEATURE "avgof14D_lag7D" ( delayed data)
* THIS REALLY HELPED US IMPROVE OUR R2 VALUE TO A RESPECTABLE LEVEL. ( WITHOUT THIS FEATURE OUR R2 value was in the 0.2 to 0.3 range)
* I reported the following R2 VALUES TO A RESPECTABLE 
 * 0.854 FOR APPL,
 * 0.87 FOR FB, 
 * 0.93 FOR AMAZON,
 * 0.85 FOR FB, AND 
 * 0.86 FOR GOOGLE.
* I RAN THE NOTE BOOK MULTIPLE TIMES TO GET THESE R2 VALUES. ( WITH DIFFERENT COMAPNIES DATAFRAME in command 11 )

##Conclusion
  
   - I have identified the best regression model that fits the dataset for Apple stock over a period of 5 years (2013 -2018)   
   - After comparing the performance of all the models, I identified Decision Tree regression model (depth 5 with 63 nodes) as the best fit model.
   - During the EDA I compared yearly performance of the Facebook, Amazon, Apple, Netflix and Google (FAANG) stocks.
   - Additionally, I compared the two high performing stocks - Apple and google and represented it graphically.
   - The key parameters have been explained using graphical representation of data. 
   - I have compared 'date' against the fields 'open', 'close', 'high' and 'low' for Apple Stock, to get a  better understanding of the fields.
   - I have fit the model with closing stock market price as the dependent variable and, volume and converted_date as the independent variables
   - During model fitting, I encountered data leakage due to high correlation between features 'close' and 'high', 'low', 'open'. Hence, we removed those features from the fit.
   - For the best model I have evaluated on a test dataset and reported the r2 value and represented it visually.

##To pass on "Lessons Learned"
  
   - Predicting future stock market prices is hard. -- some stocks are easier to predict than others
   - Splitting data based on "randomSplit" shows uncharacteristically high R2 values - use time based seperation of train and test data early on
   - Databricks community edition becomes hard to work with when number of rows is around 600,000!
   - Time series analysis helps to a certain extent. ( Adding Lag. PySpark supports "Lag" innately)
   - Adding more features is useful. #( from multiple sources)
   - Moving average of data is essential to create a model. 
   -- Model learned unneccesary details ( daily fluctuations and performed worse in test data.)
   - I could only predict as far as 7-10 days. After that the R2 values of the models started moving to less than 50 percent
   - I ran the model for all FAANG stocks